In this notebook, we try to reproduce the results from <i>Table 1</i> in <em>Yuan, Longhao & Zhao, Qibin. (2017). Completion of High Order Tensor Data with Missing
    Entries via Tensor-train Decomposition.</em>

In [0]:
# Run if using google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Setup path

import os
import sys

# From jupyter notebook, starting from Tensor-Completion/
'''
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
'''

# From google colab
# Add parent packages to path
repository_path = "/content/drive/My Drive/Tensor-Completion"
sys.path.append(repository_path)
sys.path.append(repository_path +"/Code")

# Add lib directory, containing the required libraries, to path
libs_path = "/content/drive/My Drive/lib"
sys.path.append(libs_path)

In [0]:
try:
  import cupy as cp
except:
  print("cupy not available")

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorly as tl
import time

from Models.CP_WOPT import CP_WOPT_Model
from Models.TT_WOPT import TT_WOPT_Model
from metrics import RSE
from Utils import mask_tensor

In [0]:
def run_CP_experiment(data, missing_rates, rank=10, n_epochs=10000, gtol=1e-6):
  errors = {}

  for missing_rate in missing_rates:
    # Mask the tensor
    data_masked = mask_tensor(data, missing_rate)

    # Train the model
    model = CP_WOPT_Model(data_masked, rank, init="normal", optimization="ncg")
    model.train(n_epochs, gtol)

    # Compute the RSE of our prediction
    prediction = model.predict(use_observed=False)
    error = RSE(data, prediction)

    errors[f"mr_{missing_rate}"] = error

  return errors

In [0]:
def run_TT_experiment(data, missing_rates, rank=20, n_epochs=10000, gtol=1e-6):
  n_dims = len(data.shape)
  tt_ranks = tuple([1] + [rank] * (n_dims-1) + [1])
  errors = {}

  for missing_rate in missing_rates:
    # Special case for seven-way tensor and mr of 0.95
    if missing_rate == 0.95 and n_dims == 7:
      tt_ranks = tuple([1] + [8]*(n_dims-1) + [1])
    # Mask the tensor
    data_masked = mask_tensor(data, missing_rate)

    # Train the model
    model = TT_WOPT_Model(data_masked, tt_ranks, optimization="ncg")
    model.train(n_epochs, gtol)

    # Compute the RSE of our prediction
    prediction = model.predict(use_observed=False)
    error = RSE(data, prediction)

    errors[f"mr_{missing_rate}"] = error

  return errors

<h2> 3D Synthetic data <h2>

In [0]:
# Generate 3D synthetic data

shape = (30,30,30) # Shape of our synthetic data
tensor = cp.random.normal(size=shape) # Fake tensor to create our model
model = CP_WOPT_Model(tensor, rank=10, init="normal")

# We now use our model's randomly initialized factors to create the synthetic data
factors = list(model.factors.values())
data = tl.kruskal_tensor.kruskal_to_tensor((None, factors))

In [0]:
CP_errors = run_CP_experiment(data, (0, 0.5, 0.95))

In [0]:
CP_errors

{'mr_0': 6.84052406073306e-08,
 'mr_0.5': 8.61200682850913e-08,
 'mr_0.95': 14.940322878622803}

In [0]:
TT_errors = run_TT_experiment(data, (0, 0.5, 0.95))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1040
         Function evaluations: 2277
         Gradient evaluations: 2277
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 824
         Function evaluations: 1741
         Gradient evaluations: 1741
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 105
         Function evaluations: 211
         Gradient evaluations: 211


In [0]:
TT_errors

{'mr_0': 4.396691075452397e-06,
 'mr_0.5': 5.1913383600802276e-06,
 'mr_0.95': 1.7815357713136328}

<h2>7D Synthetic Data</h2>

In [0]:
# Generate 7D synthetic data
shape = (4,4,4,4,4,4,4) # Shape of our synthetic data
tensor = cp.random.normal(size=shape) # Fake tensor to create our model
model = CP_WOPT_Model(tensor, rank=10, init="normal")

# We now use our model's randomly initialized factors to create the synthetic data
factors = list(model.factors.values())
data = tl.kruskal_tensor.kruskal_to_tensor((None, factors))

In [0]:
CP_errors = run_CP_experiment(data, (0, 0.5, 0.95))

         Current function value: 0.000000
         Iterations: 14525
         Function evaluations: 25252
         Gradient evaluations: 25240
         Current function value: 0.000000
         Iterations: 7484
         Function evaluations: 12636
         Gradient evaluations: 12624
         Current function value: 0.174394
         Iterations: 50000
         Function evaluations: 77033
         Gradient evaluations: 77033


In [0]:
CP_errors

{'mr_0': 7.102091437002834e-07,
 'mr_0.5': 7.34085903523453e-07,
 'mr_0.95': 47.78496414382384}

In [0]:
TT_errors = run_TT_experiment(data, (0, 0.5, 0.95))

         Current function value: 0.172447
         Iterations: 50000
         Function evaluations: 77459
         Gradient evaluations: 77459
         Current function value: 0.207510
         Iterations: 50000
         Function evaluations: 77026
         Gradient evaluations: 77026
         Current function value: 0.001863
         Iterations: 50000
         Function evaluations: 76560
         Gradient evaluations: 76560


In [0]:
TT_errors

{'mr_0': 0.0901922706726681,
 'mr_0.5': 0.11536077664627697,
 'mr_0.95': 1.3005824411550324}